# Vehicle Routing Problem with Time Windows (VRPTW)
## Complete Implementation with OR-Tools

**Author:** Rafael Lopes Pinheiro  
**Date:** November 23, 2025  
**Repository:** PAA-2---PROJETO-EXTRA  

This notebook provides a complete, executable implementation of the Vehicle Routing Problem with Time Windows (VRPTW) using Google OR-Tools. It demonstrates optimization techniques for route planning with capacity and time constraints.

---

## Table of Contents
1. [Setup and Installation](#setup)
2. [Import Required Libraries](#imports)
3. [Problem Overview](#overview)
4. [Data Structure and Parameters](#data)
5. [Implementation](#implementation)
6. [Running the Solver](#solver)
7. [Results and Visualization](#results)
8. [Experiments and Analysis](#experiments)

<a id='setup'></a>
## 1. Setup and Installation

First, we need to install the required packages. This cell will install Google OR-Tools (the optimization solver), along with matplotlib for visualization, numpy for numerical operations, and pandas for data handling.

**Run this cell first!**

In [ ]:
# Install required packages
# Note: You might need to restart the kernel after installation
!pip install ortools matplotlib numpy pandas

print("✅ All packages installed successfully!")

<a id='imports'></a>
## 2. Import Required Libraries

Import all necessary libraries for the optimization problem.

In [ ]:
# Import necessary libraries
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print("✅ All libraries imported successfully!")
print(f"   OR-Tools version: {pywrapcp.__version__ if hasattr(pywrapcp, '__version__') else 'installed'}")

<a id='overview'></a>
## 3. Problem Overview

### What is VRPTW?

The **Vehicle Routing Problem with Time Windows (VRPTW)** is a combinatorial optimization problem where:

- **Objective:** Find optimal routes for a fleet of vehicles to serve customers while minimizing total distance
- **Constraints:**
  - Each customer must be visited exactly once
  - Each customer has a specific time window for service
  - Each vehicle has a maximum capacity
  - All routes start and end at a central depot

### Real-World Applications:
- 📦 Package delivery (Amazon, FedEx, UPS)
- 🍕 Food delivery services
- 🚛 Waste collection and management
- 🏥 Healthcare home visits
- 🔧 Field service technician scheduling

### Problem Complexity:
VRPTW is NP-hard, meaning exact solutions become computationally infeasible for large instances. OR-Tools uses advanced heuristics and metaheuristics to find good solutions quickly.

<a id='data'></a>
## 4. Data Structure and Parameters

Let's create the data model for our problem. This includes:
- Location coordinates
- Customer demands
- Vehicle capacities
- Time windows for each location

In [ ]:
def create_data_model():
    """
    Creates the data structure for the VRPTW problem.
    
    Returns:
        dict: A dictionary containing all problem parameters
    """
    data = {}
    
    # Location coordinates (x, y) - Index 0 is the depot
    data['locations'] = [
        (456, 320),  # 0: Depot (warehouse/distribution center)
        (228, 0),    # 1: Customer 1
        (912, 0),    # 2: Customer 2
        (0, 80),     # 3: Customer 3
        (114, 80),   # 4: Customer 4
        (570, 160),  # 5: Customer 5
        (798, 160),  # 6: Customer 6
        (342, 240),  # 7: Customer 7
        (684, 240),  # 8: Customer 8
        (570, 400),  # 9: Customer 9
        (912, 400),  # 10: Customer 10
        (114, 480),  # 11: Customer 11
        (228, 480),  # 12: Customer 12
        (342, 560),  # 13: Customer 13
        (684, 560),  # 14: Customer 14
        (0, 640),    # 15: Customer 15
        (798, 640)   # 16: Customer 16
    ]
    
    # Customer demands (units) - Depot has 0 demand
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    
    # Vehicle capacities (maximum load each vehicle can carry)
    data['vehicle_capacities'] = [15, 15, 15, 15]  # 4 vehicles
    
    # Number of vehicles
    data['num_vehicles'] = 4
    
    # Depot index
    data['depot'] = 0
    
    # Time windows (start_time, end_time) for each location
    # Each customer must be visited within their time window
    data['time_windows'] = [
        (0, 5),     # 0: Depot
        (7, 12),    # 1: Customer 1
        (10, 15),   # 2: Customer 2
        (16, 18),   # 3: Customer 3
        (10, 13),   # 4: Customer 4
        (0, 5),     # 5: Customer 5
        (5, 10),    # 6: Customer 6
        (0, 4),     # 7: Customer 7
        (5, 10),    # 8: Customer 8
        (0, 3),     # 9: Customer 9
        (10, 16),   # 10: Customer 10
        (10, 15),   # 11: Customer 11
        (0, 5),     # 12: Customer 12
        (5, 10),    # 13: Customer 13
        (7, 8),     # 14: Customer 14
        (10, 15),   # 15: Customer 15
        (11, 15)    # 16: Customer 16
    ]
    
    return data

# Create the data model
data = create_data_model()

print("✅ Data model created successfully!")
print(f"   📍 Number of locations: {len(data['locations'])} (1 depot + {len(data['locations'])-1} customers)")
print(f"   🚚 Number of vehicles: {data['num_vehicles']}")
print(f"   📦 Total demand: {sum(data['demands'])} units")
print(f"   🔋 Vehicle capacity: {data['vehicle_capacities'][0]} units each")

<a id='implementation'></a>
## 5. Implementation

### 5.1 Distance Matrix Computation

We need to compute the Euclidean distance between all locations.

In [ ]:
def compute_euclidean_distance_matrix(locations):
    """
    Computes the Euclidean distance matrix between all locations.
    
    Args:
        locations: List of (x, y) tuples representing coordinates
    
    Returns:
        dict: Distance matrix where distances[i][j] is the distance from location i to j
    
    Formula: distance = sqrt((x2-x1)^2 + (y2-y1)^2)
    """
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance formula
                distances[from_counter][to_counter] = int(
                    np.hypot((from_node[0] - to_node[0]), (from_node[1] - to_node[1]))
                )
    return distances

# Compute distance matrix
distance_matrix = compute_euclidean_distance_matrix(data['locations'])
data['distance_matrix'] = distance_matrix

print("✅ Distance matrix computed successfully!")
print(f"   Example: Distance from Depot (0) to Customer 1: {distance_matrix[0][1]} units")
print(f"   Example: Distance from Customer 1 to Customer 2: {distance_matrix[1][2]} units")

### 5.2 Solution Printing Function

This function formats and displays the solution in a readable way.

In [ ]:
def print_solution(data, manager, routing, solution):
    """
    Prints the solution in a detailed, formatted way.
    
    Args:
        data: The problem data
        manager: OR-Tools index manager
        routing: OR-Tools routing model
        solution: The solution found by the solver
    """
    print("\n" + "="*70)
    print("                    SOLUTION FOUND")
    print("="*70)
    
    time_dimension = routing.GetDimensionOrDie('Time')
    total_distance = 0
    total_load = 0
    total_time = 0
    
    all_routes = []
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f"\n🚚 Route for Vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        route_details = []
        
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            node = manager.IndexToNode(index)
            route_load += data['demands'][node]
            time_min = solution.Min(time_var)
            time_max = solution.Max(time_var)
            
            route_details.append({
                'location': node,
                'load': route_load,
                'time': time_min,
                'time_window': data['time_windows'][node]
            })
            
            plan_output += f"  Location {node}"
            if node != 0:
                plan_output += f" (Demand: {data['demands'][node]})"
            plan_output += f"\n    • Time: {time_min} (window: {data['time_windows'][node][0]}-{data['time_windows'][node][1]})\n"
            plan_output += f"    • Load after visit: {route_load}/{data['vehicle_capacities'][vehicle_id]}\n"
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        
        # Add final depot return
        time_var = time_dimension.CumulVar(index)
        node = manager.IndexToNode(index)
        time_min = solution.Min(time_var)
        
        route_details.append({
            'location': node,
            'load': route_load,
            'time': time_min,
            'time_window': data['time_windows'][node]
        })
        
        plan_output += f"  Location {node} (Return to Depot)\n"
        plan_output += f"    • Arrival time: {time_min}\n"
        plan_output += f"\n  📊 Route Statistics:\n"
        plan_output += f"    • Distance: {route_distance} units\n"
        plan_output += f"    • Load: {route_load}/{data['vehicle_capacities'][vehicle_id]} units\n"
        plan_output += f"    • Duration: {time_min} time units\n"
        
        print(plan_output)
        
        all_routes.append(route_details)
        total_distance += route_distance
        total_load += route_load
        total_time += time_min
    
    print("\n" + "="*70)
    print("                  OVERALL STATISTICS")
    print("="*70)
    print(f"  🚚 Total Distance (all vehicles): {total_distance} units")
    print(f"  📦 Total Load delivered: {total_load} units")
    print(f"  ⏱️  Total Time: {total_time} time units")
    print(f"  🎯 Objective Value: {solution.ObjectiveValue()}")
    print("="*70 + "\n")
    
    return all_routes

print("✅ Solution printing function defined!")

<a id='solver'></a>
## 6. Running the Solver

This is the main function that sets up and solves the VRPTW problem. It includes:
1. Creating the routing model
2. Defining distance callbacks
3. Adding capacity constraints
4. Adding time window constraints
5. Setting search parameters
6. Solving the problem

In [ ]:
def solve_vrptw(data):
    """
    Main function to solve the VRPTW problem.
    
    Steps:
    1. Create the routing index manager
    2. Create the routing model
    3. Define the distance callback
    4. Add capacity constraints
    5. Add time window constraints
    6. Set search parameters
    7. Solve the problem
    """
    
    # === STEP 1: Create the routing index manager ===
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),  # Number of locations
        data['num_vehicles'],           # Number of vehicles
        data['depot']                   # Depot index
    )
    print("✅ Step 1: Routing manager created")
    
    # === STEP 2: Create Routing Model ===
    routing = pywrapcp.RoutingModel(manager)
    print("✅ Step 2: Routing model created")
    
    # === STEP 3: Define Distance Callback ===
    def distance_callback(from_index, to_index):
        """Returns the distance between two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    print("✅ Step 3: Distance callback registered")
    
    # === STEP 4: Add Capacity Constraints ===
    def demand_callback(from_index):
        """Returns the demand of the node."""
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity'
    )
    print("✅ Step 4: Capacity constraints added")
    
    # === STEP 5: Add Time Window Constraints ===
    def time_callback(from_index, to_index):
        """Returns the travel time between two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    
    horizon = 120  # Maximum time for a route
    routing.AddDimension(
        time_callback_index,
        horizon,  # allow waiting time
        horizon,  # maximum time per vehicle
        False,    # Don't force start cumul to zero
        'Time'
    )
    
    time_dimension = routing.GetDimensionOrDie('Time')
    
    # Add time window constraints for each location
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    # Add time window constraints for depot (start and end)
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1]
        )
    
    # Minimize the time dimension (arrival time at depot)
    for vehicle_id in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(vehicle_id))
        )
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(vehicle_id))
        )
    
    print("✅ Step 5: Time window constraints added")
    
    # === STEP 6: Set Search Parameters ===
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(30)  # 30 second time limit
    print("✅ Step 6: Search parameters configured")
    print("  - Strategy: PATH_CHEAPEST_ARC")
    print("  - Metaheuristic: GUIDED_LOCAL_SEARCH")
    print("  - Time limit: 30 seconds")
    
    # === STEP 7: Solve the Problem ===
    print("\n🔄 Solving the problem...\n")
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print("✅ Solution found!\n")
        routes = print_solution(data, manager, routing, solution)
        return manager, routing, solution, routes
    else:
        print("❌ No solution found!")
        return None, None, None, None

# Solve the problem
manager, routing, solution, routes = solve_vrptw(data)

<a id='results'></a>
## 7. Results and Visualization

### 7.1 Solution Analysis

Let's analyze the solution quality and efficiency.

In [ ]:
# Additional analysis
if solution and routes:
    print("\n=== DETAILED ANALYSIS ===")
    print(f"\n1. Feasibility Check:")
    customers_visited = len([r for route in routes for r in route if r['location'] != 0])
    total_customers = len(data['locations']) - 1
    print(f"   ✓ All customers visited: {customers_visited == total_customers} ({customers_visited}/{total_customers})")
    print(f"   ✓ All time windows respected: Yes (solver guarantees this)")
    print(f"   ✓ All capacity constraints satisfied: Yes (solver guarantees this)")
    
    print(f"\n2. Efficiency Metrics:")
    for i, route in enumerate(routes):
        customers_in_route = len([r for r in route if r['location'] != 0])
        if customers_in_route > 0:
            print(f"   Vehicle {i}: {customers_in_route} customers served")
    
    print(f"\n3. Capacity Utilization:")
    for i, route in enumerate(routes):
        max_load = route[-1]['load']
        capacity = data['vehicle_capacities'][i]
        utilization = (max_load / capacity) * 100
        print(f"   Vehicle {i}: {utilization:.1f}% ({max_load}/{capacity} units)")
else:
    print("No solution to analyze.")

### 7.2 Route Visualization

Visualize the routes on a 2D map.

In [ ]:
def plot_routes(data, routes):
    """
    Visualizes the vehicle routes on a 2D plot.
    
    Args:
        data: Problem data
        routes: Solution routes
    """
    plt.figure(figsize=(14, 12))
    
    # Colors for different vehicles
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    
    # Plot depot
    depot_x, depot_y = data['locations'][0]
    plt.plot(depot_x, depot_y, 's', markersize=20, color='black', label='Depot', zorder=5)
    plt.text(depot_x + 20, depot_y + 20, 'DEPOT', fontsize=12, fontweight='bold')
    
    # Plot routes for each vehicle
    for vehicle_id, route in enumerate(routes):
        if len(route) <= 2:  # Skip empty routes (only depot start/end)
            continue
        
        color = colors[vehicle_id % len(colors)]
        
        # Plot route path
        for i in range(len(route) - 1):
            from_loc = data['locations'][route[i]['location']]
            to_loc = data['locations'][route[i + 1]['location']]
            
            plt.plot([from_loc[0], to_loc[0]], [from_loc[1], to_loc[1]], 
                    color=color, linewidth=2, alpha=0.7, zorder=1)
        
        # Plot customer locations for this route
        route_customers = [r['location'] for r in route if r['location'] != 0]
        for customer in route_customers:
            x, y = data['locations'][customer]
            plt.plot(x, y, 'o', markersize=12, color=color, zorder=3)
            plt.text(x + 15, y + 15, str(customer), fontsize=9)
        
        # Add legend entry
        plt.plot([], [], color=color, linewidth=2, label=f'Vehicle {vehicle_id}')
    
    plt.xlabel('X Coordinate', fontsize=12)
    plt.ylabel('Y Coordinate', fontsize=12)
    plt.title('VRPTW Solution - Vehicle Routes', fontsize=16, fontweight='bold')
    plt.legend(loc='best', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot the solution
if routes:
    plot_routes(data, routes)
else:
    print("No routes to visualize.")

### 7.3 Time Window Visualization

Visualize arrival times vs. time windows for each vehicle route.

In [ ]:
def plot_time_windows(data, routes):
    """
    Visualizes the time windows and arrival times for each vehicle.
    """
    # Filter out empty routes
    non_empty_routes = [(i, route) for i, route in enumerate(routes) if len(route) > 2]
    
    if not non_empty_routes:
        print("No routes with customers to visualize.")
        return
    
    num_routes = len(non_empty_routes)
    fig, axes = plt.subplots(num_routes, 1, figsize=(14, 4 * num_routes))
    
    if num_routes == 1:
        axes = [axes]
    
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    
    for idx, (vehicle_id, route) in enumerate(non_empty_routes):
        ax = axes[idx]
        color = colors[vehicle_id % len(colors)]
        
        locations = [r['location'] for r in route]
        times = [r['time'] for r in route]
        
        # Plot time windows as horizontal bars
        for i, loc in enumerate(locations):
            tw_start, tw_end = data['time_windows'][loc]
            ax.barh(i, tw_end - tw_start, left=tw_start, height=0.4, 
                   color='lightgray', alpha=0.5, label='Time Window' if i == 0 else '')
        
        # Plot arrival times
        ax.plot(times, range(len(times)), 'o-', color=color, linewidth=2, 
               markersize=8, label='Arrival Time')
        
        # Labels
        ax.set_yticks(range(len(locations)))
        ax.set_yticklabels([f"Loc {loc}" for loc in locations])
        ax.set_xlabel('Time', fontsize=11)
        ax.set_ylabel('Location', fontsize=11)
        ax.set_title(f'Vehicle {vehicle_id} - Time Schedule', fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

# Plot time schedules
if routes:
    plot_time_windows(data, routes)
else:
    print("No routes to visualize.")

<a id='experiments'></a>
## 8. Experiments and Custom Scenarios

### 8.1 How to Modify Inputs

You can create your own custom scenarios by modifying the data model. Here's an example:

In [ ]:
def create_custom_data_model():
    """
    Example: Create your own custom dataset here!
    Modify the parameters to test different scenarios.
    """
    data = {}
    
    # Example: Simple 5-customer problem
    data['locations'] = [
        (50, 50),   # Depot
        (10, 10),   # Customer 1
        (90, 10),   # Customer 2
        (10, 90),   # Customer 3
        (90, 90),   # Customer 4
        (50, 10),   # Customer 5
    ]
    
    data['demands'] = [0, 3, 5, 2, 4, 3]
    data['vehicle_capacities'] = [10, 10]
    data['num_vehicles'] = 2
    data['depot'] = 0
    
    data['time_windows'] = [
        (0, 100),   # Depot - always available
        (0, 30),    # Customer 1 - early morning
        (0, 30),    # Customer 2 - early morning
        (30, 60),   # Customer 3 - mid morning
        (30, 60),   # Customer 4 - mid morning
        (60, 90),   # Customer 5 - late morning
    ]
    
    return data

# Uncomment the lines below to solve with custom data:
# custom_data = create_custom_data_model()
# custom_data['distance_matrix'] = compute_euclidean_distance_matrix(custom_data['locations'])
# custom_manager, custom_routing, custom_solution, custom_routes = solve_vrptw(custom_data)
# if custom_routes:
#     plot_routes(custom_data, custom_routes)
#     plot_time_windows(custom_data, custom_routes)

print("✅ Custom data model example ready!")
print("  Uncomment the code above to test with custom data")

### 8.2 Export Solution

Save the solution to a JSON file for future use or reporting.

In [ ]:
import json
from datetime import datetime

def export_solution(data, routes, solution, filename=None):
    """
    Export the solution to a JSON file.
    """
    if not routes or not solution:
        print("No solution to export.")
        return
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"results/solutions/vrptw_solution_{timestamp}.json"
    
    # Prepare solution data
    solution_data = {
        'metadata': {
            'timestamp': datetime.now().isoformat(),
            'num_vehicles': data['num_vehicles'],
            'num_customers': len(data['locations']) - 1,
            'objective_value': solution.ObjectiveValue()
        },
        'routes': []
    }
    
    for vehicle_id, route in enumerate(routes):
        if len(route) <= 2:  # Skip empty routes
            continue
        
        route_data = {
            'vehicle_id': vehicle_id,
            'stops': []
        }
        
        for stop in route:
            route_data['stops'].append({
                'location': stop['location'],
                'arrival_time': stop['time'],
                'time_window': stop['time_window'],
                'cumulative_load': stop['load']
            })
        
        solution_data['routes'].append(route_data)
    
    # Save to file
    import os
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(solution_data, f, indent=2)
    
    print(f"✅ Solution exported to: {filename}")
    return filename

# Export the current solution
if routes and solution:
    export_solution(data, routes, solution)
else:
    print("No solution to export.")

## 9. Summary and Conclusions

### What We Learned:

1. **VRPTW is a complex optimization problem** that balances multiple constraints:
   - Vehicle capacity limits
   - Time window restrictions
   - Distance minimization

2. **OR-Tools provides powerful solvers** that can find good solutions quickly using advanced heuristics

3. **Key factors affecting solution quality:**
   - Number of vehicles available
   - Vehicle capacity
   - Time window tightness
   - Customer locations and demands

### Understanding the Output:

The solution provides several key pieces of information:

1. **Route for each vehicle:** Shows the sequence of customer visits
2. **Time at each location:** When the vehicle arrives (must be within time window)
3. **Load after visit:** Cumulative load ensures capacity is not exceeded
4. **Route statistics:** Distance traveled, total load, and duration
5. **Overall statistics:** Total distance, load, and time across all vehicles

### Real-World Applications:

- **📦 Logistics:** Optimize delivery routes for package delivery companies
- **🍕 Food Delivery:** Plan efficient routes for food delivery services
- **🗑️ Waste Management:** Schedule garbage collection routes
- **🏥 Healthcare:** Plan home healthcare visits
- **🔧 Field Service:** Schedule technician visits

### Next Steps:

1. Try modifying the input data to see how it affects the solution
2. Experiment with different solver parameters
3. Add additional constraints (e.g., driver breaks, priority customers)
4. Test with larger datasets
5. Compare different solution strategies

### How to Modify Parameters:

**Change Customer Locations:**
```python
data['locations'] = [(0, 0), (10, 20), (30, 40), ...]
```

**Change Customer Demands:**
```python
data['demands'] = [0, 5, 3, 7, 2, ...]
```

**Change Vehicle Capacity:**
```python
data['vehicle_capacities'] = [20, 20, 20]
```

**Change Number of Vehicles:**
```python
data['num_vehicles'] = 3
```

**Change Time Windows:**
```python
data['time_windows'] = [(0, 10), (0, 5), (5, 10), ...]
```

---

### References:

- [Google OR-Tools Documentation](https://developers.google.com/optimization)
- [VRPTW Problem Description](https://en.wikipedia.org/wiki/Vehicle_routing_problem)
- [OR-Tools VRP Examples](https://developers.google.com/optimization/routing/vrp)

---

**Author:** Rafael Lopes Pinheiro  
**Date:** November 23, 2025  
**Repository:** [PAA-2---PROJETO-EXTRA](https://github.com/RafaelLopesPinheiro/PAA-2---PROJETO-EXTRA)

---

## 🎯 Congratulations!

You've successfully:
- ✅ Installed and configured OR-Tools
- ✅ Created a VRPTW problem instance
- ✅ Solved the problem using Google OR-Tools
- ✅ Visualized the solution
- ✅ Analyzed the results
- ✅ Exported the solution

Feel free to modify the parameters and experiment with different scenarios!